In [10]:
"""
Exhaustive CCZ-Equivalence Check for m=3 (F_8).
Checks G_a vs G_1 (Li-Kaleyski 1) and H_a vs H_1 (Li-Kaleyski 2).

*Added timing per test
"""
from sage.all import *
import numpy as np
import time

# ==========================================
# 1. Optimized Matrix Construction (Numpy)
# ==========================================

def element_to_int(elt):
    """Universal method to get integer representation of a field element."""
    return int(elt.polynomial().change_ring(ZZ)(2))

def int_to_bits_array(n, num_bits):
    """Convert integer to bit list (Little Endian)."""
    return [(n >> k) & 1 for k in range(num_bits)]

def precompute_field_data(F, m, q):
    """Pre-computes powers and bit representations."""
    elements = list(F)
    elem_to_int = {elem: element_to_int(elem) for elem in elements}
    powers_q = {elem: elem**q for elem in elements}
    powers_q1 = {elem: elem**(q+1) for elem in elements}
    elem_to_bits = {elem: int_to_bits_array(elem_to_int[elem], m) for elem in elements}
    return elements, elem_to_int, powers_q, powers_q1, elem_to_bits

def build_graph_code(func_type, F, m, q, a_val):
    """
    Builds the graph code for either G_a or H_a family.
    
    G_a: (x^{q+1} + ax^qz + yz^q, x^qz + y^{q+1}, xy^q + ay^qz + z^{q+1})
    H_a: (x^{q+1} + axy^q + yz^q, xy^q + z^{q+1}, x^qz + y^{q+1} + ay^qz)
    """
    elements, _, powers_q, powers_q1, elem_to_bits = precompute_field_data(F, m, q)
    
    # Dimensions
    n_cols = len(elements)**3
    row_size = 6 * m 
    col_matrix = np.zeros((row_size, n_cols), dtype=np.uint8)
    col_idx = 0
    
    for x in elements:
        x_bits = elem_to_bits[x]
        x_q = powers_q[x]
        x_q1 = powers_q1[x]
        for y in elements:
            y_bits = elem_to_bits[y]
            y_q = powers_q[y]
            y_q1 = powers_q1[y]
            for z in elements:
                z_bits = elem_to_bits[z]
                z_q = powers_q[z]
                z_q1 = powers_q1[z]
                
                if func_type == 'G':
                    # G_a (Li-Kaleyski 1 Type)
                    # CRITICAL FIX: Proper parentheses for field arithmetic
                    c1 = x_q1 + a_val * x_q * z + y * z_q
                    c2 = x_q * z + y_q1
                    c3 = x * y_q + a_val * y_q * z + z_q1
                    
                elif func_type == 'H':
                    # H_a (Li-Kaleyski 2 Type / Generalized F2)
                    c1 = x_q1 + a_val * x * y_q + y * z_q
                    c2 = x * y_q + z_q1
                    c3 = x_q * z + y_q1 + a_val * y_q * z

                # Fill Matrix
                col_matrix[:m, col_idx] = x_bits
                col_matrix[m:2*m, col_idx] = y_bits
                col_matrix[2*m:3*m, col_idx] = z_bits
                col_matrix[3*m:4*m, col_idx] = elem_to_bits[c1]
                col_matrix[4*m:5*m, col_idx] = elem_to_bits[c2]
                col_matrix[5*m:6*m, col_idx] = elem_to_bits[c3]
                col_idx += 1
                
    M_sage = Matrix(GF(2), col_matrix.tolist()).transpose()
    return LinearCode(M_sage)

# ==========================================
# 2. Main Logic
# ==========================================

def check_roots_Qa(a, F, q):
    """
    Checks if Q_a(T) = T^(q^2+q+1) + a*T + 1 has no roots.
    This is a necessary condition for the function to be a permutation.
    """
    P = F['T']
    T = P.gen()
    poly = T**(q**2 + q + 1) + a*T + 1
    return len(poly.roots()) == 0

def run_exhaustive_m3():
    m = 3
    print(f"{'='*70}")
    print(f"EXHAUSTIVE CCZ CHECK FOR m={m} (Field GF(2^{m}) = GF(8))")
    print(f"{'='*70}")
    
    F = GF(2**m, 'w')
    w = F.gen()
    
    # Display field elements for reference
    print(f"\nField elements (as integers):")
    for i in range(2**m):
        bits = int_to_bits_array(i, m)
        elem = F(bits)
        print(f"  {i}: {bits} -> {elem}")
    
    all_results = []
    
    # Iterate through valid q values where gcd(i, m)=1
    # For m=3: i ∈ {1, 2} gives q ∈ {2, 4}
    for i in [1, 2]:
        q = 2**i
        print(f"\n{'='*70}")
        print(f"TESTING WITH q = 2^{i} = {q}")
        print(f"{'='*70}")
        
        # 1. Build Reference Codes (a=1)
        print("\n[Phase 1] Building Reference Codes...")
        print("  - G_1 (Li-Kaleyski Type 1, a=1)...")
        start = time.time()
        C_G1 = build_graph_code('G', F, m, q, F(1))
        print(f"    Built in {time.time()-start:.2f}s")
        
        print("  - H_1 (Li-Kaleyski Type 2, a=1)...")
        start = time.time()
        C_H1 = build_graph_code('H', F, m, q, F(1))
        print(f"    Built in {time.time()-start:.2f}s")
        
        # Pre-calculate weight distributions for fast filtering
        print("\n[Phase 2] Computing reference weight distributions...")
        wd_G1 = C_G1.weight_distribution()
        wd_H1 = C_H1.weight_distribution()
        print(f"  G_1 weight dist: {wd_G1}")
        print(f"  H_1 weight dist: {wd_H1}")

        # 2. Test all non-trivial values of a
        print(f"\n[Phase 3] Testing all a ∈ F \\ {{0, 1}} ({2**m - 2} values)...")
        
        for k in range(2, 2**m):  # Skip 0 and 1
            # Convert integer k to field element
            bits = int_to_bits_array(k, m)
            a = F(bits)
            
            print(f"\n  ─────────────────────────────────────────")
            print(f"  Testing a = {a} (index {k})")
            
            # Check permutation condition
            if not check_roots_Qa(a, F, q):
                print(f"      Q_a has roots → NOT a permutation polynomial")
                print(f"      Skipping CCZ checks")
                all_results.append((i, k, a, 'G', 'SKIP-NOT-PERM', '-'))
                all_results.append((i, k, a, 'H', 'SKIP-NOT-PERM', '-'))
                continue
            
            # Check theoretical prediction (affine equivalence condition)
            is_affine = (a**(q+1) == 1)
            theory = "CCZ-Equiv (Theory)" if is_affine else "New Class (Theory)"
            print(f"     a^(q+1) = {a**(q+1)} → {theory}")
            
            # --- TEST G FAMILY ---
            print(f"\n    [G Family Test]")
            start = time.time()
            C_Ga = build_graph_code('G', F, m, q, a)
            build_time = time.time() - start
            
            # Fast filter: weight distribution
            wd_Ga = C_Ga.weight_distribution()
            if wd_Ga != wd_G1:
                result = "NOT-CCZ (Weight)"
                print(f"       NOT CCZ-Equivalent (weight dist differs)")
                print(f"       {build_time:.2f}s (build only)")
                all_results.append((i, k, a, 'G', result, build_time))
            else:
                # Slow check: isomorphism
                print(f"       Weight distributions match, checking isomorphism...")
                iso_start = time.time()
                is_ccz = C_G1.is_permutation_equivalent(C_Ga)
                iso_time = time.time() - iso_start
                total_time = build_time + iso_time
                
                if is_ccz:
                    status = " EXPECTED" if is_affine else "  SURPRISING"
                    result = "CCZ-Equiv"
                    print(f"      {status} CCZ-Equivalent")
                else:
                    result = "NOT-CCZ (Iso)"
                    print(f"       NOT CCZ but same weights (NEW CLASS?)")
                    
                print(f"       {total_time:.2f}s (build: {build_time:.2f}s, iso: {iso_time:.2f}s)")
                all_results.append((i, k, a, 'G', result, total_time))

            # --- TEST H FAMILY ---
            print(f"\n    [H Family Test]")
            start = time.time()
            C_Ha = build_graph_code('H', F, m, q, a)
            build_time = time.time() - start
            
            wd_Ha = C_Ha.weight_distribution()
            if wd_Ha != wd_H1:
                result = "NOT-CCZ (Weight)"
                print(f"       NOT CCZ-Equivalent (weight dist differs)")
                print(f"       {build_time:.2f}s (build only)")
                all_results.append((i, k, a, 'H', result, build_time))
            else:
                print(f"       Weight distributions match, checking isomorphism...")
                iso_start = time.time()
                is_ccz = C_H1.is_permutation_equivalent(C_Ha)
                iso_time = time.time() - iso_start
                total_time = build_time + iso_time
                
                if is_ccz:
                    status = " EXPECTED" if is_affine else "  SURPRISING"
                    result = "CCZ-Equiv"
                    print(f"      {status} CCZ-Equivalent")
                else:
                    result = "NOT-CCZ (Iso)"
                    print(f"       NOT CCZ but same weights (NEW CLASS?)")
                    
                print(f"        {total_time:.2f}s (build: {build_time:.2f}s, iso: {iso_time:.2f}s)")
                all_results.append((i, k, a, 'H', result, total_time))
    
    # ==========================================
    # FINAL SUMMARY
    # ==========================================
    print(f"\n\n{'='*70}")
    print("EXHAUSTIVE TEST SUMMARY")
    print(f"{'='*70}")
    print(f"{'i':<3} {'q':<3} {'a_idx':<6} {'a':<8} {'Family':<8} {'Result':<20} {'Time':<8}")
    print("-"*70)
    
    for i, k, a, fam, result, t in all_results:
        q = 2**i
        time_str = f"{t:.2f}s" if isinstance(t, float) else t
        print(f"{i:<3} {q:<3} {k:<6} {str(a):<8} {fam:<8} {result:<20} {time_str:<8}")
    
    print("="*70)
    
    # Count results by type
    g_ccz = sum(1 for r in all_results if r[3] == 'G' and 'CCZ-Equiv' in r[4])
    h_ccz = sum(1 for r in all_results if r[3] == 'H' and 'CCZ-Equiv' in r[4])
    
    print(f"\nG family: {g_ccz} CCZ-equivalent to G_1")
    print(f"H family: {h_ccz} CCZ-equivalent to H_1")

if __name__ == "__main__":
    run_exhaustive_m3()

EXHAUSTIVE CCZ CHECK FOR m=3 (Field GF(2^3) = GF(8))

Field elements (as integers):
  0: [0, 0, 0] -> 0
  1: [1, 0, 0] -> 1
  2: [0, 1, 0] -> w
  3: [1, 1, 0] -> w + 1
  4: [0, 0, 1] -> w^2
  5: [1, 0, 1] -> w^2 + 1
  6: [0, 1, 1] -> w^2 + w
  7: [1, 1, 1] -> w^2 + w + 1

TESTING WITH q = 2^1 = 2

[Phase 1] Building Reference Codes...
  - G_1 (Li-Kaleyski Type 1, a=1)...
    Built in 0.03s
  - H_1 (Li-Kaleyski Type 2, a=1)...
    Built in 0.02s

[Phase 2] Computing reference weight distributions...
  G_1 weight dist: [1, 18, 153, 816, 3060, 8568, 18564, 31824, 43758, 48620, 43758, 31824, 18564, 8568, 3060, 816, 153, 18, 1]
  H_1 weight dist: [1, 18, 153, 816, 3060, 8568, 18564, 31824, 43758, 48620, 43758, 31824, 18564, 8568, 3060, 816, 153, 18, 1]

[Phase 3] Testing all a ∈ F \ {0, 1} (6 values)...

  ─────────────────────────────────────────
  Testing a = w (index 2)
     a^(q+1) = w + 1 → New Class (Theory)

    [G Family Test]
       Weight distributions match, checking isomorphism.